**Jacobi Method**

Downloading matrix

In [1]:
import requests

matrix_id = "HB/bcsstk16"  # choose one of the IDs above
url = f"https://sparse.tamu.edu/mat/{matrix_id}.mat"
dest_filename = matrix_id.split("/")[-1] + ".mat"

# Download the file using requests
response = requests.get(url, stream=True)
response.raise_for_status()

with open(dest_filename, 'wb') as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

print(f"Downloaded {dest_filename}")

Downloaded bcsstk16.mat


Loading Matrix

In [2]:
import numpy as np
from scipy.io import loadmat
from scipy.sparse import csr_matrix

# Load the .mat file
data = loadmat("bcsstk16.mat")

# Access the matrix stored in the 'Problem' struct
A = data['Problem'][0, 0]['A']  # This is typically a sparse matrix already

# If A is not in CSR format, convert it
A_sparse = csr_matrix(A)

# Load RHS vector if available
if 'b' in data['Problem'][0, 0].dtype.names:
    b = data['Problem'][0, 0]['b'].flatten()
else:
    b = np.random.rand(A_sparse.shape[0])  # Generate dummy RHS

# Inspect matrix
print("Shape:", A_sparse.shape)
print("Non-zeros:", A_sparse.nnz)

Shape: (4884, 4884)
Non-zeros: 290378


8. Jacobi Method

    Matrix Requirement: Square, Diagonally dominant or SPD

In [3]:
import numpy as np
from scipy.sparse import csr_matrix, diags

def jacobi(A, b, x0=None, tol=1e-6, max_iter=1000):
    # Ensure CSR format
    A = csr_matrix(A)
    n = A.shape[0]
    x = np.zeros(n) if x0 is None else x0.copy()

    # Diagonal and remainder
    D = A.diagonal()
    if np.any(D == 0):
        raise ZeroDivisionError("Matrix has zero diagonal entries, Jacobi not applicable.")

    R = A - diags(D, offsets=0, format="csr")

    for it in range(max_iter):
        x_new = (b - R @ x) / D

        # Convergence check (infinity norm)
        if np.linalg.norm(x_new - x, ord=np.inf) < tol:
            residual = np.linalg.norm(b - A @ x_new)
            return x_new, it+1, residual

        x = x_new

    residual = np.linalg.norm(b - A @ x)
    return x, max_iter, residual

# Example usage with a diagonally dominant test matrix
from scipy.sparse import diags
n = 10
A_sparse = diags([4*np.ones(n), -1*np.ones(n-1), -1*np.ones(n-1)], [0, -1, 1], format="csr")
b = np.ones(n)

x, iterations, residual = jacobi(A_sparse, b)
print("Jacobi solution (first 5 entries):", x[:5])
print("Iterations:", iterations)
print("Final residual norm:", residual)

Jacobi solution (first 5 entries): [0.36602437 0.46409779 0.49036738 0.49737255 0.49912382]
Iterations: 19
Final residual norm: 2.578411398933292e-06
